<a href="https://colab.research.google.com/github/RamiIssa2/NLP_Project-Multi_Document_Summarization/blob/main/NLP_Multi_Document_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

colab link: https://colab.research.google.com/drive/1IOr0PNzYLourFkEAO2yi6_XynP8Jbnvg?usp=sharing

Aveen Hussein, Rami Issa , Somar Mohammad

# 1- Morphological analysis option for English language multi-document summarization:


## Lemmatization vs. Stemming vs. Light stemmer
|criteria |Lemmatization | Stemming|Light stemmer|
|:---|:------------|:--------------|---------:|
|    Speed |time consuming  |        fast |faster|
|    accuracy |most accurate   |        accurate |Less accurate|
|   Level of semantic processing|high   |       low|very low|


<!-- |   Performance |yes   |          no || -->
#  Morphological analysis option for Arabic language multi-document summarization:

## Alkhalil vs. Buckwalt vs. Light stemmer
|criteria |Alkhalil | Buckwalt|Light stemmer|
|:---|:------------|:--------------|---------:|
|    Speed |time consuming  |        fast |faster|
|    accuracy |most accurate   |        accurate |Less accurate|
|   Level of semantic processing|high   |       low|very low|
<!-- |   Morphological categories |time consuming  |        fast |cusom| -->




# 2-3- For our project there are 2 challanges:

 * the first one is identifying the most important sentences in each document and that should be done by a fast option because of the large size of the dataset when dealing with several documents. so for Arabic we have Buckwalt and for English we have Stemming (or light stemming but it's less accurate and may cause problems).  
 * the second one is identifying the similar sentences from several documents and finding the right order to place them in the summary. This challange requires higher accuracy and a higher level of semantic processing. So for Arabic we have Alkhalil and for English we have Lemmatization.


# 4- Yes, the pipeline is as follows:

1.   Sentence segmantation: we separate the sentences in each document.
2.   Word tokenization: we split each sentence into its words.
3.   N-gram: we create an N-gram of words to generate a dataset that helps with identifying the important words and expressions based on the frequency of their occurance.
4.   Morphological Analysis: Two step analysis as explained in question 2-3, 
we calculate the importance of each stem and lemma and include that in the importance of the derived words and the sentences that contain them.
5.   Sentences are compaired based on importance within each document or multiple documents and the most important sentences are chosen keeping in mind that similar sentences(based on stems and lemmas occurence and order) to the chosen ones are neglected.



# 5- Code for first stage

## importing libraries

In [ ]:
!pip install stanza
!pip install unrar
!pip install fasttext
!pip install danlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=38d78e9b66c537ff68404c47327b581ee78866af046dfc13c35e0d903111a530
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fa

In [ ]:
import nltk
import pandas as pd
import numpy as np
import sys
import os
import stanza
import math
import fasttext
import re
import json
import requests
import spacy
import danlp.models
from danlp.models import load_spacy_model
from sklearn.decomposition import LatentDirichletAllocation
from spacy import displacy
from nltk import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize , PunktSentenceTokenizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
nltk.download('state_union')
nltk.download('averaged_perceptron_tagger')

nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Getting text files

In [ ]:
!gdown 1ONvixWmlnKi3U5y0EViOlj0K03G8A8Fp

Downloading...
From: https://drive.google.com/uc?id=1ONvixWmlnKi3U5y0EViOlj0K03G8A8Fp
To: /content/texts and models.rar
100% 755k/755k [00:00<00:00, 136MB/s]


In [ ]:
!unrar x "/content/texts and models.rar"


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/texts and models.rar

Creating    texts and models                                          OK
Extracting  texts and models/lid.176.ftz                                  99%  OK 
Creating    texts and models/texts                                    OK
Creating    texts and models/texts/ar                                 OK
Extracting  texts and models/texts/ar/text1.txt                           99%  OK 
Extracting  texts and models/texts/ar/text2.txt                           99%  OK 
Creating    texts and models/texts/en                                 OK
Extracting  texts and models/texts/en/text1.txt                           99%  OK 
Extracting  texts and models/texts/en/text2.txt                           99%  OK 
All OK


## Reading text files

In [ ]:
def read_file(filename):
	try:
		with open(filename, 'r') as f:
			data = f.read()
		return data
	except IOError:
		print("Error opening or reading input file: ", filename)
		sys.exit()

In [ ]:
def read_all_files(path):
  file_texts = []
  files = os.listdir(path)
  for file_path in files:
    text = read_file(path + '/' + file_path)
    file_texts.append(text)
  return file_texts

In [ ]:
file_texts = read_all_files("/content/texts and models/texts/en")
file_texts[0]

'the dolphins are healthy.\nthe water condition is good.\nthe dolphins are being fed well.\nthe tanks are large enough to keep the dolphins happy and active.\n\nMany zoos and aquariums offer adoption of their animals as a way to offset the costs of food and veterinary bills. Some packages may include an adoption certificate, photo, and/or a plush toy. Prices for these packages range widely and are generally tax-deductible.\n\n\nAnimals at zoos or aquariums might have more than one adoptive "parent" due to the popularity of the species. You can do this by visiting the zoo in person or by looking at the establishment\'s website. Usually, you\'ll be asked to choose the dolphin you like the most out of several, select a package, and then make a payment.\n\nIf you adopt a dolphin from a local zoo or aquarium, you have the bonus of being able to visit them in person whenever you want to check up on them. Seeing the subject of your donation up close can be rewarding; people tend to respond mo

In [ ]:
file_texts[1]

'If you can spot Flipper, you can spot a Bottlenose Dolphin!\n\nThe Bottlenose Dolphin may be spotted anywhere.\n\nGrey, primarily, Bottlenose Dolphins have a stubby beak and relatively long flippers.\nTerehu is the Maori word for Bottlenose Dolphin.'

## Defining the language model classes

- English Language Model

In [ ]:
class English_Language_Model:
  def word_tokenize(self, article):
    return nltk.tokenize.word_tokenize(article)

  def stop_words(self):
    return nltk.corpus.stopwords.words()

  def sent_tokenize(self, article):
    return nltk.tokenize.sent_tokenize(article)
  
  def lemmatization(self, text):
    nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma')
    doc = nlp(text)

    lemmas = [word.lemma for sent in doc.sentences for word in sent.words]
    return lemmas

  def pos_tagging(self, text):
    nlp = stanza.Pipeline(lang='en', processors='tokenize,pos')
    doc = nlp(text)

    pos_tags = [word.pos for sent in doc.sentences for word in sent.words]
    return pos_tags
  
  def get_pos_importances(self, text):
    pos_tags = self.pos_tagging(text)
    pos_importances = []
    for tag in pos_tags:
      if tag == 'NOUN':
        pos_importances.append(2)
      elif tag == 'VERB':
        pos_importances.append(1.9)
      elif tag == 'ADV':
        pos_importances.append(1.3)
      elif tag == 'ADJ':
        pos_importances.append(1.25)
      else:
        pos_importances.append(1)
      
    return pos_importances
      
  def get_ner_importnace(self,text):
    NER = spacy.load("en_core_web_sm")
    text1= NER(text)
    ner_importances = []
    words=self.word_tokenize(text)
    
    for word in words:
      is_entity=False
      for NER_word in text1.ents:
        if (NER_word.text==word):
          is_entity=True
      if (is_entity):      
        ner_importances.append(1.5)
        
      else:
        ner_importances.append(1)
    return ner_importances
  def get_dependency_importnace(self,text):
   words=self.word_tokenize(text)
   nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')
   doc = nlp(text)

    
    # Printing dependency features for each token 
   dependency_importances = [1 for i in doc.sentences[0].words]


   for token in doc.sentences[0].words:

     dependency_importances[token.head-1]+=0.1
   return dependency_importances
  def get_chunks_importance(self,text):
   words = nltk.word_tokenize(text)
   tagged = nltk.pos_tag(words)
   chunks_importances = []
   chunkGram = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
   chunkParser = nltk.RegexpParser(chunkGram)
   chunked = chunkParser.parse(tagged)
   print(chunked)

   for subtree in chunked.subtrees():
       for x in subtree:
         if (subtree.label() == 'Chunk'):
           chunks_importances.append(1.5)
         else:
           chunks_importances.append(1)
   return chunks_importances


- Arabic Language Model

In [ ]:
class Arabic_Language_Model:
  def word_tokenize(self, article):
    return nltk.tokenize.word_tokenize(article)

  def stop_words(self):
    return nltk.corpus.stopwords.words('arabic')

  def sent_tokenize(self, article):
    article = re.sub("؟", "?", article)
    return nltk.tokenize.sent_tokenize(article)

  def lemmatization(self, text):
    url = 'https://farasa.qcri.org/webapi/lemmatization/'
    api_key = "kusYKWEbCaQFtSumEA"
    payload = {'text': text, 'api_key': api_key}
    data = requests.post(url, data=payload)
    lemmas = json.loads(data.text)
    return lemmas['text']

  def pos_tagging(self, text):
    nlp = stanza.Pipeline(lang='ar', processors='tokenize,pos')
    doc = nlp(text)

    pos_tags = [word.pos for sent in doc.sentences for word in sent.words]
    return pos_tags
  
  def get_pos_importances(self, text):
    pos_tags = self.pos_tagging(text)
    pos_importances = []
    for tag in pos_tags:
      if tag == 'NOUN':
        pos_importances.append(2)
      elif tag == 'VERB':
        pos_importances.append(1.9)
      elif tag == 'ADV':
        pos_importances.append(1.3)
      elif tag == 'ADJ':
        pos_importances.append(1.25)
      else:
        pos_importances.append(1)
      
    return pos_importances
  def get_ner_importnace(self,text):
    # nlp = spacy.blank('ar')
    # ner = nlp.create_pipe('ner')
    # nlp.add_pipe(ner)
    # # NER = spacy.load("ar_core_web_sm")
    # text1= nlp(text)
    # ner_importances = []
    # words=self.word_tokenize(text)
    
    # for word in words:
    #   is_entity=False
    #   for NER_word in text1.ents:
    #     if (NER_word.text==word):
    #       is_entity=True
    #   if (is_entity):      
    #     ner_importances.append(1.5)
        
    #   else:
    #     ner_importances.append(1)
     ###because we did not find an acceptable arabic ner tool then we will neglect the effect of ner
     nlp = stanza.Pipeline(lang='ar', processors='tokenize,mwt,pos,lemma,depparse')
     doc = nlp(text)

       
       # Printing dependency features for each token 
     ner_importances = [1 for i in doc.sentences[0].words]
     return ner_importances
  def get_dependency_importnace(self,text):
    ###because we did not find an acceptable arabic dependancy tool then we will neglect the effect of dependancy trees
   nlp = stanza.Pipeline(lang='ar', processors='tokenize,mwt,pos,lemma,depparse')
   doc = nlp(text)

    
    # Printing dependency features for each token 
   dependency_importances = [1 for i in doc.sentences[0].words]


  #  for token in doc.sentences[0].words:

  #    dependency_importances[token.head-1]+=0.1
   return dependency_importances
  def get_chunks_importance(self,text):
    #  words = nltk.word_tokenize(text)
    #  tagged = nltk.pos_tag(words)
    #  chunks_importances = []
    #  chunkGram = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
    #  chunkParser = nltk.RegexpParser(chunkGram)
    #  chunked = chunkParser.parse(tagged)
    #  print(chunked)
    
    #  for subtree in chunked.subtrees():
    #      for x in subtree:
    #        if (subtree.label() == 'Chunk'):
    #          chunks_importances.append(1.5)
    #        else:
    #          chunks_importances.append(1)
     ###because we did not find an acceptable arabic chunking tool then we will neglect the effect of chunking
     nlp = stanza.Pipeline(lang='ar', processors='tokenize,mwt,pos,lemma,depparse')
     doc = nlp(text)

       
       # Printing dependency features for each token 
     chunks_importances = [1 for i in doc.sentences[0].words]
     return chunks_importances

## Detecting the language

> Indented block



In [ ]:
def remove_new_lines(article):
  new_article = ""
  for letter in article:
    if letter != '\n':
      new_article += letter

  return new_article

In [ ]:
model = fasttext.load_model('/content/texts and models/lid.176.ftz')
lang1 = model.predict(remove_new_lines(file_texts[0]), k=1)
lang2 = model.predict(remove_new_lines(file_texts[1]), k=1)
if lang1[0][0] == lang2[0][0]:
  if lang1[0][0] == '__label__en':
    print("English")
    Lang_Model = English_Language_Model()
  elif lang1[0][0] == '__label__ar':
    print("Arabic")
    Lang_Model = Arabic_Language_Model()
  else:
    print("Unknown language detected")
else:
  print("The two articles are written in different languages")

English


## Removing stop words

In [ ]:
def remove_stop_words(articles):
  new_articles = []
  for article in articles:
    text_tokens = Lang_Model.word_tokenize(article)
    tokens_without_sw = [word for word in text_tokens if not word in Lang_Model.stop_words()]
    tokens_without_sw = [word for word in tokens_without_sw if not word in [',', '،']]

    new_article = ""
    for token in tokens_without_sw:
      new_article += token + ' '
    new_articles.append(new_article)

  return new_articles

In [ ]:
articles_with_sw = file_texts
articles = remove_stop_words(file_texts)

## Sentence Segmentation

In [ ]:
article_sentences_with_sw = []
for article in articles_with_sw:
  article_sentences_with_sw.append([sentence.strip() for sentence in Lang_Model.sent_tokenize(article)])

article_sentences = []
for article in articles:
  article_sentences.append([sentence.strip() for sentence in Lang_Model.sent_tokenize(article)])

print(article_sentences)

tmp_article_sentences = []
for article in article_sentences:
  tmp = []
  for sent in article:
    sent_tokens = Lang_Model.word_tokenize(sent)
    sent_tokens = [word for word in sent_tokens if word != '.']
    new_sent = ""
    for token in sent_tokens:
      new_sent += token + ' '
    tmp.append(new_sent)
  tmp_article_sentences.append(tmp)

article_sentences = tmp_article_sentences

[['dolphins healthy .', 'water condition .', 'dolphins fed .', 'tanks large dolphins happy active .', 'Many zoos aquariums offer adoption animals offset costs food veterinary bills .', 'Some packages include adoption certificate photo and/or plush toy .', 'Prices packages range widely generally tax-deductible .', "Animals zoos aquariums adoptive `` parent '' due popularity species .", "You visiting zoo person establishment 's website .", "Usually 'll asked choose dolphin select package make payment .", 'If adopt dolphin local zoo aquarium bonus visit person check .', 'Seeing subject donation close rewarding ; tend respond charitable identifiable recipient .', 'You introduce family friends sponsored dolphin .', 'There ample research suggest captivity detrimental dolphins cetaceans .', 'Dolphins highly intelligent emotional creatures live complex societies wild travel move regularly ; theory tank big accommodate dolphins swim 100 miles day wild .', 'Keep mind adopting dolphin zoo aquariu

# Word Tokenization

In [ ]:
def get_lemmas(text):
  return Lang_Model.lemmatization(text)

In [ ]:
def get_pos_tags(text):
  return Lang_Model.pos_tagging(text)

In [ ]:
def get_pos_tags_importance(text):
  return Lang_Model.get_pos_importances(text)

In [ ]:
def get_ner_importnace(text):
  return Lang_Model.get_ner_importnace(text)


In [ ]:
def get_dependency_importnace(text):
  return Lang_Model.get_dependency_importnace(text)


In [ ]:
def computeTf(lemmas, lemmas_total):
  fdist = nltk.FreqDist(lemmas)
  lemmas_Tf = {}
  for k, v in fdist.items():
    lemmas_Tf[k[0]] = v / lemmas_total
  return lemmas_Tf

In [ ]:
def computeIdf(lemma, articles):
  N = len(articles)
  nidf = 0
  for article in articles:
      if (lemma in (get_lemmas(article))):
        nidf += 1
  result = math.log(N / nidf)
  return result

In [ ]:
def get_chunks_importance(text):
  return Lang_Model.get_chunks_importance(text)
    



In [ ]:
def get_lemmas_importance(articles):
  lemmas_importance = []
  for article in articles: 
    lemmas_in_doc = get_lemmas(article)
    lemmas_total = len(lemmas_in_doc)
    lemma_doc = ""
    
    for lemma in lemmas_in_doc:
      lemma_doc += lemma + ' '
    lemmas = ngrams(lemma_doc.split(), 1)
    
    lemmas_Tf = computeTf(lemmas, lemmas_total)

    for lemma in lemmas:
      lemmas_Tf[lemma] *= computeIdf(lemma, articles)
    lemmas_importance.append(lemmas_Tf)
  
  return lemmas_importance

In [ ]:
lemmas_importance = get_lemmas_importance(articles)
print(lemmas_importance[0])

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


{'dolphin': 0.057803468208092484, 'healthy': 0.005780346820809248, '.': 0.09826589595375723, 'water': 0.005780346820809248, 'condition': 0.005780346820809248, 'feed': 0.005780346820809248, 'tank': 0.011560693641618497, 'large': 0.005780346820809248, 'happy': 0.005780346820809248, 'active': 0.005780346820809248, 'many': 0.005780346820809248, 'zoo': 0.03468208092485549, 'aquarium': 0.028901734104046242, 'offer': 0.005780346820809248, 'adoption': 0.011560693641618497, 'animal': 0.023121387283236993, 'offset': 0.005780346820809248, 'cost': 0.005780346820809248, 'food': 0.005780346820809248, 'veterinary': 0.005780346820809248, 'bill': 0.005780346820809248, 'some': 0.005780346820809248, 'package': 0.017341040462427744, 'include': 0.005780346820809248, 'certificate': 0.005780346820809248, 'photo': 0.005780346820809248, 'and': 0.005780346820809248, '/': 0.005780346820809248, 'or': 0.005780346820809248, 'plush': 0.005780346820809248, 'toy': 0.005780346820809248, 'price': 0.005780346820809248, '

In [ ]:
def get_sent_importance(sent, article_num):
  lemmas = get_lemmas(sent)
  pos_importances = get_pos_tags_importance(sent)
  ner_importances = get_ner_importnace(sent)
  dependency_importnaces=get_dependency_importnace(sent)
  chunk_importnaces=get_chunks_importance(sent)
  print(sent, article_num)
  for lemma in lemmas:
    print(lemma)
  result = sum([lemmas_importance[article_num][lemma] * importance*ner_importance*chunk_importnace for lemma in lemmas
                for importance in pos_importances
                for ner_importance in ner_importances
                for dependency_importnace in dependency_importnaces
                for chunk_importnace in chunk_importnaces
                ])
  return result

In [ ]:
article_lemma_sentences = []
articale_sent_importance = []
index = 0
for article in article_sentences:
  article_lemma_sentences.append([])
  articale_sent_importance.append([])
  for sent in article:
    lemmas = get_lemmas(sent)
    article_lemma_sentences[index].append(lemmas)
    s=get_sent_importance(sent, index)
    articale_sent_importance[index].append(get_sent_importance(sent, index))
  index += 1

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


...

(S
  (Chunk Terehu/NNP Maori/NNP word/NN)
  (Chunk Bottlenose/NNP Dolphin/NNP))
Terehu Maori word Bottlenose Dolphin  1
terehu
Maori
word
bottlenose
dolphin


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| lemma     | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


(S
  (Chunk Terehu/NNP Maori/NNP word/NN)
  (Chunk Bottlenose/NNP Dolphin/NNP))
Terehu Maori word Bottlenose Dolphin  1
terehu
Maori
word
bottlenose
dolphin


In [ ]:
print(lemmas_importance[0])
print(article_lemma_sentences[0][1])
print(articale_sent_importance[0][1])

{'dolphin': 0.057803468208092484, 'healthy': 0.005780346820809248, '.': 0.09826589595375723, 'water': 0.005780346820809248, 'condition': 0.005780346820809248, 'feed': 0.005780346820809248, 'tank': 0.011560693641618497, 'large': 0.005780346820809248, 'happy': 0.005780346820809248, 'active': 0.005780346820809248, 'many': 0.005780346820809248, 'zoo': 0.03468208092485549, 'aquarium': 0.028901734104046242, 'offer': 0.005780346820809248, 'adoption': 0.011560693641618497, 'animal': 0.023121387283236993, 'offset': 0.005780346820809248, 'cost': 0.005780346820809248, 'food': 0.005780346820809248, 'veterinary': 0.005780346820809248, 'bill': 0.005780346820809248, 'some': 0.005780346820809248, 'package': 0.017341040462427744, 'include': 0.005780346820809248, 'certificate': 0.005780346820809248, 'photo': 0.005780346820809248, 'and': 0.005780346820809248, '/': 0.005780346820809248, 'or': 0.005780346820809248, 'plush': 0.005780346820809248, 'toy': 0.005780346820809248, 'price': 0.005780346820809248, '

In [ ]:
temp_article_sentences = []
article_index = 0
for article in article_sentences:
  tmp = []
  sent_index = 0
  for sent in article:
    tmp.append((article_sentences_with_sw[article_index][sent_index], sent, articale_sent_importance[article_index][sent_index]))
    sent_index += 1
  temp_article_sentences.append(tmp)
  article_index += 1

article_sentences = temp_article_sentences

In [ ]:
summarization_factor = 0.7

In [ ]:
def sort_sentences(article):
 lst = len(article)
 for i in range(0, lst):       
      for j in range(0, lst - i - 1):
          if (article[j][2] < article[j + 1][2]):
              temp = article[j]
              article[j] = article[j + 1]
              article[j + 1] = temp
 return article

In [ ]:
def is_similar(lemma_sentence1, lemma_sentence2):
  lemmas = [lemma_sentence1, lemma_sentence2]
  lemmas1 = Lang_Model.word_tokenize(min(lemmas, key=len))
  lemmas2 = Lang_Model.word_tokenize(max(lemmas, key=len))

  count = 0
  for lemma1 in lemmas1:
      if lemmas2.count(lemma1) >= 1:
        count+=1
  similar_ratio = count / len(lemmas1)
  if similar_ratio > 0.8:
    return True, max(lemmas, key = len)
  else: return False, ""


In [ ]:
def get_k_sentences_in_article(summarization_factor, article):


  temp = []
  k = int(len(article) * summarization_factor)
  sorted_sentences=sort_sentences(article)
  
  temp.append(sorted_sentences[0])
  for sent in sorted_sentences[1:k]:
    new_sent = True
    for y in temp:
      c, t = is_similar(sent[1], y[1]) 

      if c:
        new_sent = False

    if new_sent:
      temp.append(sent)
  return temp, k

In [ ]:
def get_k_sentences(summarization_factor, article_sentences):
  k_sentences = [] 
  lenn = 0
  old_k = 0
  for article in article_sentences:
    temp, temp_k = get_k_sentences_in_article(summarization_factor, article)
    k_sentences.append(temp)
    lenn += len(temp)
    old_k = max(temp_k,old_k)
  # print (old_k)
  k = int(lenn * summarization_factor)

  result = [k_sentences[0][0]]

  for i in range (0, old_k):
      new_sent = True
      for art_index in range(0, len(article_sentences)):
        if(i<len(k_sentences[art_index])):
          for y in result:
              c, t = is_similar(k_sentences[art_index][i][1], y[1]) 
              # print(c)
              if  c:
                new_sent = False

          if new_sent:
            # print(k_sentences[art_index][i], art_index, i)
            result.append(k_sentences[art_index][i])
          if len(result) >= k:
            return result
  return result

      



In [ ]:
final_text = get_k_sentences(summarization_factor, article_sentences)

In [ ]:
final_summary = ""
for test in final_text:
  final_summary += test[0] + '\n'

# **Write the summary in a text file**

In [ ]:
with open('/content/summary.txt', 'w') as f:
    f.write(final_summary)


**Key phrase Extraction:** The goal of this step is to recognize key phrases from the sentences to help reviewers quickly identify parts of the sentence that are relevant to the extraction goal. Based on the type of data element we employed the following technique:
Noun phrase chunking and regex matching: For literal-expression data elements (e.g., outcome), applying regular expression matching might detect very long phrases, which is less useful for key phrase identification. Therefore, we performed noun phrase chunking to restrict the matching only to noun phrases of the sentence. To perform the noun phrase chunking, we used the Stanford parser to generate a Penn tree and used the Tregex parser [35] to collect all noun phrase expressions.
source: https://www.sciencedirect.com/science/article/pii/S1532046416301514

# Another Approach: Latent Dirichlet Allocation (LDA)
Finding Optimal Number of Topics using GridSearch to maximize LDA Maximum Likelihood

In [ ]:
!pip install sklearn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=0c8ae892649f163f3a7b703006333454271f4b10d41b302be462e97070537098
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [ ]:
import sklearn.model_selection
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [ ]:
def get_topics(articles=articles):
  doctopic={}
  models = []
  doctopic[0] = {}
  doctopic[1] = {}
  doctopic[2] = {}
  doctopic[3] = {}
  topics_scores=np.zeros(4)
  for article, article_with_sw in articles, articles_with_sw:
    sentences=Lang_Model.sent_tokenize(article)
    cv = CountVectorizer(max_features=10000, ngram_range=(1, 2), max_df=0.98)
    cv_X = cv.fit_transform(sentences)
    lda = LatentDirichletAllocation(n_components=4)
    models.append(lda.fit_transform(cv_X))
  for model in models:
    for sentence_score in model:

        topics_scores+=sentence_score

  for i in range(len(articles_with_sw)):
    sentences=Lang_Model.sent_tokenize(articles_with_sw[i])
    #sent_topics={}
    for j in range(len(sentences)):
      topic_index = np.where(models[i][j] == np.max(models[i][j]))[0][0]
      doctopic[topic_index][sentences[j]] = np.max(models[i][j])
      #doctopic[topic_index].append(sent_topics[sentences[j]])
  
  return doctopic,topics_scores

In [ ]:
sent_topics,topics_scores=get_topics(articles)


we define a function to get a summary with lda

In [ ]:
def get_k_sentences_with_lda(summarization_factor, article_sentences,sent_topics, topics_scores):
  k_sentences = [] 
  final_sum=[]
  lenn = 0
  old_k = 0
  for article in article_sentences:
    temp, temp_k = get_k_sentences_in_article(summarization_factor, article)
    k_sentences.append(temp)
    lenn += len(temp)
    old_k = max(temp_k,old_k)
  # print (old_k)
  k = int(lenn * summarization_factor)
  # we add a certain number of each topic based on topic importance score and the number of sentence that should be in the 
  # summary based on the summarization factor
  num_sent_topics=[math.ceil(k* topic_score/np.sum(topics_scores)) for topic_score in topics_scores]

  for index in sent_topics:
    sorted_d=sorted(sent_topics[index],reverse=True)

    final_sum.append(sorted_d[:num_sent_topics[index]])

  return final_sum



In [ ]:
final_text_with_lda = get_k_sentences_with_lda(summarization_factor, article_sentences,sent_topics,topics_scores)

In [ ]:
final_text_with_lda

[['You can also introduce family and friends to your sponsored dolphin this way.',
  'There is ample research to suggest that being in captivity is detrimental for dolphins and other cetaceans.'],
 ['the tanks are large enough to keep the dolphins happy and active.',
  'Terehu is the Maori word for Bottlenose Dolphin.',
  'Some packages may include an adoption certificate, photo, and/or a plush toy.'],
 ['the dolphins are healthy.', 'the dolphins are being fed well.'],
 ['the water condition is good.',
  "You can do this by visiting the zoo in person or by looking at the establishment's website."]]

In [ ]:
final_summary = ""
for test in final_text_with_lda:
  for t in test:
    final_summary += t + '\n'

In [ ]:
with open('/content/summary_with_lda.txt', 'w') as f:
    f.write(final_summary)